<style>
    .title_div {
    padding: 60px;
    text-align: center;
    background: #ffffff;
    }
    #title {
        color: #f2a900;
        font-size: 80px;
    }
    #subtitle {
        color: #4d4d4e;
        font-size: 20px;
    }
</style>

<div class="title_div">
  <h1 id="title">TX</h1>
  <p id="subtitle">Transactions and Script</p>
</div>

<div style="color:red; text-align: center; padding: 60px">
⚠ The following TX examples are just to see how Bitcoin work; don't transmit nothing generated here on the mainnet. ⚠
</div>


## TOC:
* [Hashes](#hasehs)
    * [Hash256](#hash256)
    * [Hash160](#hash160)
* [VarInt](#varint)
* [TxIn](#txin)
* [TxOut](#txout)
* [TX](#tx)
* [Script](#script)
    * [Data encoding](#data-encoding)
    * [OP Codes](#op-codes)
    * [P2PK](#p2pk)
        * [ScriptPubKey](#script-pub-key-p2pk)
        * [ScriptSig](#script-sig-p2pk)
    * [P2PKH](#p2pkh)
        * [ScriptPubKey](#script-pub-key-p2pkh)
        * [ScriptSig](#script-sig-p2pkh)
    * [P2MS](#p2ms)
        * [ScriptPubKey](#script-pub-key-p2ms)
        * [ScriptSig](#script-sig-p2ms)
    * [P2SH](#p2sh)
        * [ScriptPubKey](#script-pub-key-p2sh)
        * [ScriptSig](#script-sig-p2sh)
* [Tests](#tests)

In [ ]:
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, PrivateFormat
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives.asymmetric import utils

from io import StringIO, BytesIO
from hashlib import sha256, new
import unittest
import base58
import copy

<a class="anchor" id="hashes"></a>
## Hashes

<a class="anchor" id="hash256"></a>
### Hash256

In [ ]:
def hash256(data):
    return sha256(sha256(data).digest()).digest()

<a class="anchor" id="hash160"></a>
### Hash160

In [ ]:
def hash160(data):
    return new('ripemd160', sha256(data).digest()).digest()

<a class="anchor" id="varint"></a>
## VarInt

In [ ]:
def read_varint(s):
    
    flag = s.read(1)
    if flag == 0xfd:
        # 0xfd next two bytes are the number
        return flag.read(2)
    elif flag == 0xfe:
        # 0xfe next four bytes are the number
        return flag.read(4)
    elif flag == 0xff:
        # 0xff next eight bytes are the number
        return flag.read(8)
    else:
        # the flag is the number
        return flag


def encode_varint(i):
   
    if i < 0xfd:
        return bytes([i])
    elif i < 0x10000:
        return b'\xfd' + i.to_bytes(2, 'litte')
    elif i < 0x100000000:
        return b'\xfe' + i.to_bytes(4, 'litte')
    elif i < 0x10000000000000000:
        return b'\xff' + i.to_bytes(8, 'litte')
    else:
        raise ValueError('integer too large: {}'.format(i))

<a class="anchor" id="txin"></a>
## TxIn

In [ ]:
class TxIn:
    def __init__(self, txID, outID, scriptSig = bytes(b''), sequence = bytes(b'\xff\xff\xff\xff')):
        self.txID = txID # hex string
        self.outIndex = outID # int
        self.scriptSig = scriptSig # 
        self.sequence = sequence
    
    def serialize(self):
        serialized_tx = self.txID[::-1]
        serialized_tx += self.outIndex[::-1]
        if self.scriptSig != None:
            serialized_tx += encode_varint(len(self.scriptSig))
            serialized_tx += self.scriptSig
        serialized_tx += self.sequence[::-1]

        return serialized_tx


    @classmethod
    def parse(cls, input, hex=False):
        if hex:
            stream = input
        else:
            stream = BytesIO(bytes.fromhex(input))
        TxID = stream.read(32)[::-1]
        index = stream.read(4)[::-1]
        script_size = int.from_bytes(read_varint(stream), 'little')
        script = stream.read(script_size)
        secuence = stream.read(4)[::-1]
        return cls(TxID, index, script, secuence)

    def __repr__(self):
        string = '\t[\n\t'
        string += f'TxID: {self.txID.hex()}\n\t'
        string += f'OutIndex: {self.outIndex.hex()}\n\t'
        if self.scriptSig != None:
            string += f'Script size: {len(self.scriptSig)}\n\t'
            string += f'ScriptSig: {self.scriptSig.hex()}\n\t'
        else:
            string += f'Script size: {None}\n\t'
            string += f'ScriptSig: {self.scriptSig}\n\t'
        string += f'Sequence: {self.sequence.hex()}\n' + '\t]'
        return string

    def colors(self):
        result = ''
        result += '\033[94m' + self.txID[::-1].hex()
        result += '\033[91m' + self.outIndex[::-1].hex()
        if self.scriptSig != None:
            result += '\033[93m' + encode_varint(len(self.scriptSig)).hex()
            result += '\033[96m' + self.scriptSig.hex()
        result += '\033[95m' + self.sequence[::-1].hex()
        result += '\n\033[94m\u2589 TxID'
        result += '\n\033[91m\u2589 Index'
        result += '\n\033[93m\u2589 Script Size'
        result += '\n\033[96m\u2589 ScriptSig'
        result += '\n\033[95m\u2589 Sequence'

        return result

In [ ]:
tx_in_0 = 'b674693a82cdcf1060186c92e530d280120c53952fd8b9cf66adff6b1e1ea76b010000006a47304402203c390e403858b3d4683ecd857675278e70b1c7af2a45fbe316328b0bb45c0f13022041c05ef823d6f86af1d69c9359dddbae209005451f7d3de3eba65f5b4620d951012103b577a59ddc57ce173c058c66586064809fe88cd15dd9516614f1e8e5550de5f1ffffffff'
tx_in_1 = '71dd3443b4ebbd256d7090e540d471c0b696de5fceb30f2f65ea020245733f14010000006a47304402200be80959e1e9a031810b41934e667cd883af0a42bace12314d759ac173ed8c0f02204d3b05ee9177624facaaf86971eaa25b9362c95c3c8e90fd8cf039dd3bb71dce012102294f00ee94128a71d134fede09001463e46c447cb1683e2b122e9ea323ef5319ffffffff'
tx_in_2 = '0e1e6f6008f2a46e4dc069d2a51e95ec10295f7832c48934cb0b46bf495627dd000000006a47304402202f7e4d368d50f4cf72a07922ded69e7aa8f5c106da57136130ccff48bfc7ed56022038eddfbc42e53800870b476998606c5c2f26a71950e55d79ef1337cc64b6a002012103e25df6966fe91e3bb7656b2b833d1ced860a22ac72c4ee1fbbefbed735918ca8ffffffff'
tx_in_3 = '3da3e5197ddb87a8f025b5bc230ae040c2c776dd804475619bb43eb1fc985b13010000006b483045022100b0c7736d2a6f9c87b88c49289cf485962c511211d86e93e307cd03ca6ae69bac02203bb06660c13e228c261bb9b9396129c26336232138711622735523880a4748340121032ec518e302bd1a865292817db538fb3e75bca805ed16a02c7df5d9cf4126357fffffffff'

tx_in_0 = 'b674693a82cdcf1060186c92e530d280120c53952fd8b9cf66adff6b1e1ea76b010000006a47304402203c390e403858b3d4683ecd857675278e70b1c7af2a45fbe316328b0bb45c0f13022041c05ef823d6f86af1d69c9359dddbae209005451f7d3de3eba65f5b4620d951012103b577a59ddc57ce173c058c66586064809fe88cd15dd9516614f1e8e5550de5f1ffffffff'
tx_in_1 = '71dd3443b4ebbd256d7090e540d471c0b696de5fceb30f2f65ea020245733f14010000006a47304402200be80959e1e9a031810b41934e667cd883af0a42bace12314d759ac173ed8c0f02204d3b05ee9177624facaaf86971eaa25b9362c95c3c8e90fd8cf039dd3bb71dce012102294f00ee94128a71d134fede09001463e46c447cb1683e2b122e9ea323ef5319ffffffff'
tx_in_2 = '0e1e6f6008f2a46e4dc069d2a51e95ec10295f7832c48934cb0b46bf495627dd000000006a47304402202f7e4d368d50f4cf72a07922ded69e7aa8f5c106da57136130ccff48bfc7ed56022038eddfbc42e53800870b476998606c5c2f26a71950e55d79ef1337cc64b6a002012103e25df6966fe91e3bb7656b2b833d1ced860a22ac72c4ee1fbbefbed735918ca8ffffffff'
tx_in_3 = '3da3e5197ddb87a8f025b5bc230ae040c2c776dd804475619bb43eb1fc985b13010000006b483045022100b0c7736d2a6f9c87b88c49289cf485962c511211d86e93e307cd03ca6ae69bac02203bb06660c13e228c261bb9b9396129c26336232138711622735523880a4748340121032ec518e302bd1a865292817db538fb3e75bca805ed16a02c7df5d9cf4126357fffffffff'

print(TxIn.parse(tx_in_0).colors())
print(TxIn.parse(tx_in_1).colors())
print(TxIn.parse(tx_in_2).colors())
print(TxIn.parse(tx_in_3).colors())

<a class="anchor" id="txout"></a>
## TxOut

In [ ]:
class TxOut:

    def __init__(self, value, scriptPubKey):
        self.value = value
        self.scriptPubKey = scriptPubKey

    def serialize(self):
        serialized_tx = self.value[::-1]
        serialized_tx += encode_varint(len(self.scriptPubKey))
        serialized_tx += self.scriptPubKey
        
        return serialized_tx

    @classmethod
    def parse(cls, output, hex=False):
        if hex:
            stream = output
        else:
            stream = BytesIO(bytes.fromhex(output))
        value = stream.read(8)[::-1]
        script_size = int.from_bytes(read_varint(stream), 'little')
        script = stream.read(script_size)
        return cls(value, script)

    def __repr__(self):
        string = '\t[\n\t'
        string += f'Value: {int.from_bytes(self.value, "big")}\n\t'
        string += f'Script size: {len(self.scriptPubKey)}\n\t'
        string += f'ScriptPubKey: {self.scriptPubKey.hex()}\n' + '\t]'
        return string

    def colors(self):
        result = ''
        result += '\033[91m' + self.value[::-1].hex()
        result += '\033[93m' + encode_varint(len(self.scriptPubKey)).hex()
        result += '\033[96m' + self.scriptPubKey.hex()
        result += '\n\033[91m\u2589 Value'
        result += '\n\033[93m\u2589 Script Size'
        result += '\n\033[96m\u2589 ScriptPubKey'
        return result


In [ ]:
tx_out = '880d0100000000001976a914751b5e9c9cf3ea616fc4f62f1827da9597c67dda88ac'
cash_back = '200b2000000000001976a914070f7ab537f647f048c2d1cd89dc7dfed0e6384988ac'


print(TxOut.parse(tx_out).colors())
print(TxOut.parse(cash_back).colors())

<a class="anchor" id="tx"></a>
## TX

In [ ]:
class TX:
    def __init__(self, inputs, outputs, version=b'\x00\x00\x00\x01' , locktime=b'\x00\x00\x00\x00'):
        self.inputs = inputs
        self.outputs = outputs
        self.version = version
        self.locktime = locktime

    def serialize(self):
        serialized_tx = self.version[::-1]
        serialized_tx += encode_varint(len(self.inputs))
        for i in self.inputs:
            serialized_tx += i.serialize()
        
        serialized_tx += encode_varint(len(self.outputs))
        for i in self.outputs:
            serialized_tx += i.serialize()

        serialized_tx += self.locktime[::-1]
        return serialized_tx

    @classmethod
    def parse(cls, tx):
        stream = BytesIO(bytes.fromhex(tx))

        version = stream.read(4)[::-1]

        input_count = int.from_bytes(read_varint(stream), 'little')
        inputs = []
        for i in range(input_count):
            inputs.append(TxIn.parse(stream, True))

        output_count = int.from_bytes(read_varint(stream), 'little')
        outputs = []
        for i in range(output_count):
            outputs.append(TxOut.parse(stream, True))
        locktime = stream.read(4)[::-1]

        return cls(inputs, outputs, version, locktime)
    
    def __repr__(self):
        string = ''
        string += f'Version: {int.from_bytes(self.version, "big")}\n'
        string += f'Input count: {len(self.inputs)}\n'
        string += '{\n'
        for input in self.inputs:
            string += input.__repr__() + ',\n'
        string += '}\n'
        string += f'Output count: {len(self.outputs)}\n'
        string += '{\n'
        for output in self.outputs:
            string += output.__repr__() + ',\n'
        string += '}\n'
        string += f'Locktime: {int.from_bytes(self.locktime, "big")}\n'
        return string

    def colors(self):
        result = ''
        result += '\033[94m' + self.version[::-1].hex()

        result += '\033[93m' + encode_varint(len(self.inputs)).hex()
        result += '\033[96m'
        for i in self.inputs:
            result += i.serialize().hex()
        
        result += '\033[93m' + encode_varint(len(self.outputs)).hex()
        result += '\033[95m'
        for i in self.outputs:
            result += i.serialize().hex()

        result += '\033[92m' + self.locktime[::-1].hex()

        result += '\n\033[94m\u2589 Version'
        result += '\n\033[93m\u2589 Counter'
        result += '\n\033[96m\u2589 Inputs'
        result += '\n\033[95m\u2589 Outputs'
        result += '\n\033[92m\u2589 Sequence'

        return result

In [ ]:
transaction = TX.parse('0100000004b674693a82cdcf1060186c92e530d280120c53952fd8b9cf66adff6b1e1ea76b010000006a47304402203c390e403858b3d4683ecd857675278e70b1c7af2a45fbe316328b0bb45c0f13022041c05ef823d6f86af1d69c9359dddbae209005451f7d3de3eba65f5b4620d951012103b577a59ddc57ce173c058c66586064809fe88cd15dd9516614f1e8e5550de5f1ffffffff71dd3443b4ebbd256d7090e540d471c0b696de5fceb30f2f65ea020245733f14010000006a47304402200be80959e1e9a031810b41934e667cd883af0a42bace12314d759ac173ed8c0f02204d3b05ee9177624facaaf86971eaa25b9362c95c3c8e90fd8cf039dd3bb71dce012102294f00ee94128a71d134fede09001463e46c447cb1683e2b122e9ea323ef5319ffffffff0e1e6f6008f2a46e4dc069d2a51e95ec10295f7832c48934cb0b46bf495627dd000000006a47304402202f7e4d368d50f4cf72a07922ded69e7aa8f5c106da57136130ccff48bfc7ed56022038eddfbc42e53800870b476998606c5c2f26a71950e55d79ef1337cc64b6a002012103e25df6966fe91e3bb7656b2b833d1ced860a22ac72c4ee1fbbefbed735918ca8ffffffff3da3e5197ddb87a8f025b5bc230ae040c2c776dd804475619bb43eb1fc985b13010000006b483045022100b0c7736d2a6f9c87b88c49289cf485962c511211d86e93e307cd03ca6ae69bac02203bb06660c13e228c261bb9b9396129c26336232138711622735523880a4748340121032ec518e302bd1a865292817db538fb3e75bca805ed16a02c7df5d9cf4126357fffffffff02880d0100000000001976a914751b5e9c9cf3ea616fc4f62f1827da9597c67dda88ac200b2000000000001976a914070f7ab537f647f048c2d1cd89dc7dfed0e6384988ac00000000')
print(transaction.colors())

<a class="anchor" id="script"></a>
## Script

<a class="anchor" id="data-encoding"></a>
### Data encoding

In [ ]:
def data_encoding(data):
    count = len(data)
    if count < 76:
        return count.to_bytes(1, 'little') + data
    elif count < 2**8:
       return b'\x4a' + count.to_bytes(1, 'little') + data #OP_PUSHDATA1
    elif count < 2**16: # cap at 520
        return b'\x4d' + count.to_bytes(2, 'little') + data #OP_PUSHDATA2
    elif count < 2**32:
        return b'\x4e' + count.to_bytes(4, 'little') + data #OP_PUSHDATA4

def data_decoding(data):
    code = int.from_bytes(data.read(1), 'little')
    if code == 0x4a:
        size = int.from_bytes(data.read(1), 'little') #OP_PUSHDATA1
        return data.read(size)
    elif code == 0x4d:
        size = int.from_bytes(data.read(2), 'little') #OP_PUSHDATA2
        return data.read(size)
    elif code == 0x4e:
        size = int.from_bytes(data.read(4), 'little') #OP_PUSHDATA4
        return data.read(size)
    else:
        return data.read(code)

<a class="anchor" id="op-codes"></a>
### OP Codes

In [ ]:
OP_CODES = {
    'OP_CHECKSIG': b'\xac',
    'OP_EQUAL': b'\x87',
    'OP_HASH160': b'\xa9',
    'OP_EQUALVERIFY': b'\x88',
    'OP_CHECKMULTISIG': b'\xae',
    'OP_DUP': b'\x76',
    'OP_0': b'\x00'
}

for i, code in enumerate(range(81,97)):
    name = f'OP_{i+1}'
    OP_CODES[name] = code.to_bytes(1, 'big')


for i in OP_CODES:
    print(i, end='')
    print(": 0x"+OP_CODES[i].hex())

<a class="anchor" id="p2pk"></a>
### P2PK

<a class="anchor" id="script-pub-key-p2pk"></a>
#### ScriptPubKey

In [ ]:
def p2pk_scriptPubKey(pub_key):
    return data_encoding(pub_key) + OP_CODES['OP_CHECKSIG']

<a class="anchor" id="script-sig-p2pk"></a>
#### ScriptSig

In [ ]:
def p2pk_scriptSig(sig):
    return data_encoding(sig)

<a class="anchor" id="p2pkh"></a>
### P2PKH

<a class="anchor" id="script-pub-key-p2pkh"></a>
#### ScriptPubKey

In [ ]:
def p2pkh_scriptPubKey(pub_key):
    hashed_key = hash160(pub_key)
    return OP_CODES['OP_DUP'] + OP_CODES['OP_HASH160'] + data_encoding(hashed_key) + OP_CODES['OP_EQUALVERIFY'] + OP_CODES['OP_CHECKSIG']

<a class="anchor" id="script-sig-p2pkh"></a>
#### ScriptSig

In [ ]:
def p2pkh_scriptSig(sig, pub_key):
    return data_encoding(sig) + data_encoding(pub_key)

<a class="anchor" id="p2ms"></a>
### P2MS

<a class="anchor" id="script-pub-key-p2ms"></a>
#### ScriptPubKey

In [ ]:
def p2ms_scriptPubKey(key_array, n_sig):
    result = b''
    result += OP_CODES[f'OP_{n_sig}']
    for key in key_array:
        result += data_encoding(key)
    result += OP_CODES[f'OP_{len(key_array)}']
    
    return result + OP_CODES['OP_CHECKMULTISIG']

<a class="anchor" id="script-sig-p2ms"></a>
#### ScriptSig

In [ ]:
def p2ms_scriptSig(sig_array):
    result = b''
    for sig in sig_array:
        result += data_encoding(sig)
    return OP_CODES['OP_0'] + result

<a class="anchor" id="p2sh"></a>
### P2SH

<a class="anchor" id="script-pub-key-p2sh"></a>
#### ScriptPubKey

In [ ]:
def p2sh_scriptPubKey(script):
    result = data_encoding(hash160(script))    
    return OP_CODES['OP_HASH160'] + result + OP_CODES['OP_EQUAL']

<a class="anchor" id="script-sig-p2sh"></a>
#### ScriptSig

In [ ]:
def p2sh_scriptSig(sig_script, lock_script):
    return sig_script + data_encoding(lock_script)

<a class="anchor" id="creating-tx"></a>
## Creating a transaction

In [ ]:
def wif_decode(key):
    key = base58.b58decode_check(key)
    """
    checksum = key[-4:]
    key = key[:-4]
    if hash256(key)[0:4] != checksum:
        raise Exception("Wrong WIF key.")
    """
    # Taking away network bytes
    key = key[1:]
    # Taking away compression byte
    key = key[:-1]
    return key

In [ ]:
# Inputs
in_0 = TxIn(bytes.fromhex('6ba71e1e6bffad66cfb9d82f95530c1280d230e5926c186010cfcd823a6974b6'), bytes(b'\x00\x00\x00\x01'))
in_1 = TxIn(bytes.fromhex('143f73450202ea652f0fb3ce5fde96b6c071d440e590706d25bdebb44334dd71'), bytes(b'\x00\x00\x00\x01'))
in_2 = TxIn(bytes.fromhex('dd275649bf460bcb3489c432785f2910ec951ea5d269c04d6ea4f208606f1e0e'), bytes(b'\x00\x00\x00\x00'))
in_3 = TxIn(bytes.fromhex('135b98fcb13eb49b61754480dd76c7c240e00a23bcb525f0a887db7d19e5a33d'), bytes(b'\x00\x00\x00\x01'))
input_list = [in_0, in_1, in_2, in_3]

# Output
lock_script_0 = p2pkh_scriptPubKey(bytes.fromhex('03024bbfa47bd639ebf6ed77a785175b6b3224a1547aa7d1401f95e84a6263b7f0'))
amount_0 = 69000
out_0 = TxOut(amount_0.to_bytes(8, 'big'), lock_script_0)

#Cash-back
lock_script_1 = p2pkh_scriptPubKey(bytes.fromhex('02278c58a944431b425875b60cdb229b1195923843a83f915779a451fae85fe63c'))
amount_1 = 2100000
cash_back = TxOut(amount_1.to_bytes(8, 'big'), lock_script_1)

output_list = [out_0, cash_back]

# Transaction
lock_time = 0
tx = TX(input_list, output_list, b'\x00\x00\x00\x01', lock_time.to_bytes(4, 'big'))

# Variables needed
signatures = []
locking_scripts = [
    bytes.fromhex('76a914d73ed0e5506a1b9c4c82fcc4c8d2705d308a1c7988ac'),
    bytes.fromhex('76a9143721cfdffb37ea053c8507c26ee68b92b44ce4d788ac'),
    bytes.fromhex('76a91421337c36b4d44802ed2379b55b2b66a3433fd87588ac'),
    bytes.fromhex('76a9142f6849a431d91d44ba958b0a14c5d9e22222b14888ac')

]

public_keys = []
private_wifs = [
    'cPnztkJGDnweFad2TXjqXdM1r7EBFNQUQ1XoiEfsy8mfED5kaQib',
    'cPrSqshyySHqtF4ZE8ttk4TW7w6ck88jTQfeU415ngTPq9RWrSPs',
    'cRHXZkGZeevh8SbCXYLHcRwsGx1FmuKBLMe8h3LT4M9oYUUiDKqg',
    'cScqWqFzrWhiK3ndTLpKFPuE5kCzi7degAWeqTEJaqWZTCopbAoV'
]

# Cleaning scriptSigs
for input in tx.inputs:
    input.scriptSig = b''

# Signing each input
for wif, lock_s, input_index in zip(private_wifs, locking_scripts, range(len(tx.inputs))):
    # Creating a copy to work on
    clean_tx = copy.deepcopy(tx)
    # Adding scriptPubKey to the scriptSig field as placeholder
    clean_tx.inputs[input_index].scriptSig = lock_s
    # Decoding private key
    secret_num = wif_decode(wif) # private key wif
    private_key = ec.derive_private_key(int.from_bytes(secret_num, 'big'), ec.SECP256K1())
    # Signing
    chosen_hash = hashes.SHA256()
    digest = hash256(clean_tx.serialize() + b'\x01\x00\x00\x00')# Adding hash code
    sig = private_key.sign(
        digest,
        ec.ECDSA(utils.Prehashed(chosen_hash))
    )
    # Adding SigScripts to each input
    signatures.append(sig + b'\x01')# Hash code after digest
   
    public_keys.append(private_key.public_key().public_bytes(Encoding.X962, PublicFormat.CompressedPoint))

# Comment this following lines generate new signatures
if True:
    example_sigs = ['304402203c390e403858b3d4683ecd857675278e70b1c7af2a45fbe316328b0bb45c0f13022041c05ef823d6f86af1d69c9359dddbae209005451f7d3de3eba65f5b4620d95101',
                    '304402200be80959e1e9a031810b41934e667cd883af0a42bace12314d759ac173ed8c0f02204d3b05ee9177624facaaf86971eaa25b9362c95c3c8e90fd8cf039dd3bb71dce01',
                    '304402202f7e4d368d50f4cf72a07922ded69e7aa8f5c106da57136130ccff48bfc7ed56022038eddfbc42e53800870b476998606c5c2f26a71950e55d79ef1337cc64b6a00201',
                    '3045022100b0c7736d2a6f9c87b88c49289cf485962c511211d86e93e307cd03ca6ae69bac02203bb06660c13e228c261bb9b9396129c26336232138711622735523880a47483401']
    signatures = [bytes.fromhex(i) for i in example_sigs]

# Adding SigScripts to each input
for sig, pub_key, tx_input in zip(signatures, public_keys, tx.inputs):
    tx_input.scriptSig = p2pkh_scriptSig(sig,pub_key)


print('TxID: '+hash256(tx.serialize())[::-1].hex())
print(tx.serialize().hex())



<a class="anchor" id="test"></a>
## Tests

In [ ]:
class TestTX(unittest.TestCase):
    def test_TxIn(self):
        input='7967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc010000006a47304402206a2eb16b7b92051d0fa38'\
            'c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21d'\
            'c8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825ffffffff'
        aux = TxIn.parse(input)
        output = aux.serialize().hex()
        self.assertEqual(input, output)

    def test_TxOut(self):
        input='4baf2100000000001976a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac'
        aux = TxOut.parse(input)
        output = aux.serialize().hex()
        self.assertEqual(input, output)

    def test_TX(self):
        input='01000000017967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc010000006a47304402206a2eb16b7b9'\
            '2051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122'\
            'bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825ffffffff014baf210000000'\
            '0001976a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac00000000'
        aux = TX.parse(input)
        output = aux.serialize().hex()
        self.assertEqual(input, output)

    def test_data_encoding(self):
        test = bytes(b'Hello World!')
        byte_count = len(test)
        encoded_data = data_encoding(test)
        self.assertEqual(byte_count, encoded_data[0])

    def test_data_decoding(self):
        test = bytes(b'Hello World!')
        encoded_data = data_encoding(test)
        encoded_data = encoded_data * 2
        decoded_data = data_decoding(BytesIO(encoded_data))
        self.assertEqual(test, decoded_data)
    
    ### P2PK

    def test_p2pk_scriptPubKey(self):
        key = bytes.fromhex('04ae1a62fe09c5f51b13905f07f06b99a2f7159b2225f374cd378d71302fa28414e7aab37397f554a7df5f142c21c1b7303b8a0626f1baded5c72a704f7e6cd84c')
        result = '4104ae1a62fe09c5f51b13905f07f06b99a2f7159b2225f374cd378d71302fa28414e7aab37397f554a7df5f142c21c1b7303b8a0626f1baded5c72a704f7e6cd84cac'
        self.assertEqual(p2pk_scriptPubKey(key).hex(), result)

    def test_p2pk_scriptSig(self):
        sig = bytes.fromhex('30440220576497b7e6f9b553c0aba0d8929432550e092db9c130aae37b84b545e7f4a36c022066cb982ed80608372c139d7bb9af335423d5280350fe3e06bd510e695480914f01')
        result = '4730440220576497b7e6f9b553c0aba0d8929432550e092db9c130aae37b84b545e7f4a36c022066cb982ed80608372c139d7bb9af335423d5280350fe3e06bd510e695480914f01'
        self.assertEqual(p2pk_scriptSig(sig).hex(), result)

    ### P2PKH

    def test_p2pkh_scriptPubKey(self):
        key = bytes.fromhex('044d05240cfbd8a2786eda9dadd520c1609b8593ff8641018d57703d02ba687cf2f187f0cee2221c3afb1b5ff7888caced2423916b61444666ca1216f26181398c')
        result = '76a91412ab8dc588ca9d5787dde7eb29569da63c3a238c88ac'
        self.assertEqual(p2pkh_scriptPubKey(key).hex(), result)

    def test_p2pkh_scriptSig(self):
        key = bytes.fromhex('044d05240cfbd8a2786eda9dadd520c1609b8593ff8641018d57703d02ba687cf2f187f0cee2221c3afb1b5ff7888caced2423916b61444666ca1216f26181398c')
        sig = bytes.fromhex('304502203f004eeed0cef2715643e2f25a27a28f3c578e94c7f0f6a4df104e7d163f7f8f022100b8b248c1cfd8f77a0365107a9511d759b7544d979dd152a955c867afac0ef78601 ')
        result = '48304502203f004eeed0cef2715643e2f25a27a28f3c578e94c7f0f6a4df104e7d163f7f8f022100b8b248c1cfd8f77a036510'\
            '7a9511d759b7544d979dd152a955c867afac0ef7860141044d05240cfbd8a2786eda9dadd520c1609b8593ff8641018d57703d02ba6'\
            '87cf2f187f0cee2221c3afb1b5ff7888caced2423916b61444666ca1216f26181398c'
        self.assertEqual(p2pkh_scriptSig(sig, key).hex(), result)

    ### P2MS

    def test_p2ms_scriptPubKey(self):
        key_1 = bytes.fromhex('04cc71eb30d653c0c3163990c47b976f3fb3f37cccdcbedb169a1dfef58bbfbfaff7d8a473e7e2e6d317b87bafe8bde97e3cf8f065dec022b51d11fcdd0d348ac4')
        key_2 = bytes.fromhex('0461cbdcc5409fb4b4d42b51d33381354d80e550078cb532a34bfa2fcfdeb7d76519aecc62770f5b0e4ef8551946d8a540911abe3e7854a26f39f58b25c15342af')
        output = p2ms_scriptPubKey([key_1, key_2], 1).hex()
        correct_output = '514104cc71eb30d653c0c3163990c47b976f3fb3f37cccdcbedb169a1dfef58bbfbfaff7d8a473e7e2e6d317b87baf'\
            'e8bde97e3cf8f065dec022b51d11fcdd0d348ac4410461cbdcc5409fb4b4d42b51d33381354d80e550078cb532a34bfa2fcfdeb7d76'\
            '519aecc62770f5b0e4ef8551946d8a540911abe3e7854a26f39f58b25c15342af52ae'
        self.assertEqual(output, correct_output)

    def test_p2ms_scriptSig(self):
        sig = bytes.fromhex('304402203f16c6f40162ab686621ef3000b04e75418a0c0cb2d8aebeac894ae360ac1e780220ddc15ecdfc3507ac48e1681a33eb60996631bf6bf5bc0a0682c4db743ce7ca2b01')
        output = p2ms_scriptSig([sig]).hex()
        correct_output = '0047304402203f16c6f40162ab686621ef3000b04e75418a0c0cb2d8aebeac894ae360ac1e780220ddc15ecdfc3507ac48e1681a33eb60996631bf6bf5bc0a0682c4db743ce7ca2b01'
        self.assertEqual(output, correct_output)

    ### P2SH

    def test_p2sh_scriptPubKey(self):
        key_1 = bytes.fromhex('022afc20bf379bc96a2f4e9e63ffceb8652b2b6a097f63fbee6ecec2a49a48010e')
        key_2 = bytes.fromhex('03a767c7221e9f15f870f1ad9311f5ab937d79fcaeee15bb2c722bca515581b4c0')
        script = p2ms_scriptPubKey([key_1, key_2], 1)
        output = p2sh_scriptPubKey(script).hex()
        correct_output = 'a914748284390f9e263a4b766a75d0633c50426eb87587'
        self.assertEqual(output, correct_output)

    def test_p2sh_scriptSig(self):
        # pubKey script
        key_1 = bytes.fromhex('022afc20bf379bc96a2f4e9e63ffceb8652b2b6a097f63fbee6ecec2a49a48010e')
        key_2 = bytes.fromhex('03a767c7221e9f15f870f1ad9311f5ab937d79fcaeee15bb2c722bca515581b4c0')
        lock_script = p2ms_scriptPubKey([key_1, key_2], 1)
        # Sig script
        sig = bytes.fromhex('3046022100a07b2821f96658c938fa9c68950af0e69f3b2ce5f8258b3a6ad254d4bc73e11e022100e82fab8df3f7e7a28e91b3609f91e8ebf663af3a4dc2fd2abd954301a5da67e701')
        sig_script = p2ms_scriptSig([sig])
        output = p2sh_scriptSig(sig_script, lock_script).hex()
        correct_output = '00493046022100a07b2821f96658c938fa9c68950af0e69f3b2ce5f8258b3a6ad254d4bc73e11e022100e82fab8df3f7'\
            'e7a28e91b3609f91e8ebf663af3a4dc2fd2abd954301a5da67e701475121022afc20bf379bc96a2f4e9e63ffceb8652b2b6a097f63fbe'\
            'e6ecec2a49a48010e2103a767c7221e9f15f870f1ad9311f5ab937d79fcaeee15bb2c722bca515581b4c052ae'
        self.assertEqual(output, correct_output)


if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)